# o1 Style Thinking

In this recipe, we will show how to achieve Chain-of-Thought Reasoning.
This makes LLMs to breakdown the task in multiple steps and generate a coherent output allowing to solve complex tasks in logical steps.

<div class="admonition tip">
<p class="admonition-title">Mirascope Concepts Used</p>
<ul>
<li><a href="../../../learn/prompts/">Prompts</a></li>
<li><a href="../../../learn/calls/">Calls</a></li>
<li><a href="../../../learn/response_models/">Response Models</a></li>
</ul>
</div>

<div class="admonition note">
<p class="admonition-title">Background</p>
<p>
    Large Language Models (LLMs) are known to generate text that is coherent and fluent. However, they often struggle with tasks that require multi-step reasoning or logical thinking. In this recipe, we will show how to use Mirascope to guide the LLM to break down the task into multiple steps and generate a coherent output.

</p>
</div>


## Setup

To set up our environment, first let's install all of the packages we will use:





In [ ]:
!pip install "mirascope[groq]" 
!pip install datetime

In [ ]:
# Set the appropriate API key for the provider you're using
# Here we are using GROQ_API_KEY

export GROQ_API="Your API Key"

# Without Chain-of-Thought Reasoning

We will begin by showing how a typical LLM performs on a task that requires multi-step reasoning. In this example, we will ask the model to generate a count the number of `s`s in the word `Mississssippi` (Yes it has 7`s`'s). We will use the `llama-3.1-8b-instant` for this example.

In [2]:
from datetime import datetime

from mirascope.core import groq

history: list[dict[str, str]] = []


@groq.call("llama-3.1-8b-instant")
def generate_answer(question: str) -> str:
    return f"Generate an answer to this question: {question}"


def run() -> None:
    question: str = "how many s's in the word mississssippi"
    response: str = generate_answer(question)
    print(f"(User): {question}")
    print(f"(Assistant): {response}")
    history.append({"role": "user", "content": question})
    history.append({"role": "assistant", "content": response})


run()

(User): how many s's in the word mississssippi
(Assistant): There are 5 s's in the word 'mississippi'.


In this example, the zero-shot method is used to generate the output. The model is not provided with any additional information or context to help it generate the output. The model is only given the input prompt and asked to generate the output.

This is not so effective when there is a logcial task to be performed.

Now let's see how the model performs on this task when it can reason using Chain-of-Thought Reasoning.

# With Chain of Thought Reasoning

In [43]:
from mirascope.core import groq


history: list[dict] = []

MODEL = "llama-3.1-8b-instant"


@groq.call(MODEL)
def cot_step(prompt: str, step_number: int, previous_steps: str) -> str:
    return f"""
    You are an expert AI assistant that explains your reasoning step by step. For this step, provide a title that describes what you're doing, along with the content. Decide if you need another step or if you're ready to give the final answer.

    USE AS MANY REASONING STEPS AS POSSIBLE. AT LEAST 3. BE AWARE OF YOUR LIMITATIONS AS AN LLM AND WHAT YOU CAN AND CANNOT DO. IN YOUR REASONING, INCLUDE EXPLORATION OF ALTERNATIVE ANSWERS. CONSIDER YOU MAY BE WRONG, AND IF YOU ARE WRONG IN YOUR REASONING, WHERE IT WOULD BE. FULLY TEST ALL OTHER POSSIBILITIES. YOU CAN BE WRONG. WHEN YOU SAY YOU ARE RE-EXAMINING, ACTUALLY RE-EXAMINE, AND USE ANOTHER APPROACH TO DO SO. DO NOT JUST SAY YOU ARE RE-EXAMINING. USE AT LEAST 3 METHODS TO DERIVE THE ANSWER. USE BEST PRACTICES.

    IMPORTANT: DO NOT USE CODE BLOCKS OR PROGRAMMING EXAMPLES IN YOUR REASONING. EXPLAIN YOUR PROCESS IN PLAIN LANGUAGE.

    This is step number {step_number}.

    Question: {prompt}

    Previous steps:
    {previous_steps}

    Respond in the following format:
    Step {step_number}: [Title]
    [Content]
    Next action: [continue/final_answer]
    """


@groq.call(MODEL)
def final_answer(prompt: str, reasoning: str) -> str:
    return f"""
    Based on the following chain of reasoning, provide a final answer to the question. Only provide the text response without any titles or preambles. Retain any formatting as instructed by the original prompt, such as exact formatting for free response or multiple choice.

    Question: {prompt}

    Reasoning:
    {reasoning}

    Final Answer:
    """


def generate_cot_response(
    user_query: str,
) -> tuple[list[tuple[str, str, float]], float]:
    steps: list[tuple[str, str, float]] = []
    total_thinking_time: float = 0.0
    step_count: int = 1
    reasoning: str = ""
    previous_steps: str = ""

    while True:
        start_time: datetime = datetime.now()
        step_result: str = cot_step(user_query, step_count, previous_steps).content
        end_time: datetime = datetime.now()
        thinking_time: float = (end_time - start_time).total_seconds()

        lines = step_result.strip().split("\n")
        title = lines[0].split(": ", 1)[1]
        content = "\n".join(lines[1:-1])
        next_action = lines[-1].split(": ", 1)[1]

        steps.append((f"Step {step_count}: {title}", content, thinking_time))
        total_thinking_time += thinking_time

        reasoning += f"\n{step_result}\n"
        previous_steps += f"\n{step_result}\n"

        if next_action.lower() == "final_answer" or step_count >= 5:
            break

        step_count += 1

    # Generate final answer
    start_time = datetime.now()
    final_result: str = final_answer(user_query, reasoning).content
    end_time = datetime.now()
    thinking_time = (end_time - start_time).total_seconds()
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_result, thinking_time))

    return steps, total_thinking_time


def display_cot_response(
    steps: list[tuple[str, str, float]], total_thinking_time: float
) -> None:
    for title, content, thinking_time in steps:
        print(f"{title}:")
        print(content.strip())
        print(f"**Thinking time: {thinking_time:.2f} seconds**\n")

    print(f"**Total thinking time: {total_thinking_time:.2f} seconds**")


def run() -> None:
    question: str = "How many s's are in the word 'mississssippi'?"
    print("(User):", question)
    # Generate COT response
    steps, total_thinking_time = generate_cot_response(question)
    display_cot_response(steps, total_thinking_time)

    # Add the interaction to the history
    history.append({"role": "user", "content": question})
    history.append(
        {"role": "assistant", "content": steps[-1][1]}
    )  # Add only the final answer to the history


# Run the function

run()

(User): How many s's are in the word 'mississssippi'?
Step 1: Identifying the Task and Clarifying the Question:
The task requires me to determine the number of 's' surnames in the word 'mississssippi'. However, the terminology 'surnames' might be misleading here since we are talking about the word 'mississssippi', a word that consists of multiple letters and not names.

Assuming the correct interpretation of the task is to count the number of 's' occurrences in the word 'mississssippi', it seems we are dealing with a relatively straightforward task. The first step would be:

1.1: Breaking down the word 'mississssippi' into individual letters to visually inspect and manipulate the letters. By doing this, we can then observe the number of times the letter 's' repeats.

Upon visual inspection of the letters in 'mississssippi', the word contains:

m-i-s-s-i-s-s-s-s-s-i-p-p-i

1.2: Observing the consistency of my visual inspection of the letters by reading them together. Analyzing each char

As demonstrated in the COT Reasoning example, we can guide the model to break down the task into multiple steps and generate a coherent output. This allows the model to solve complex tasks in logical steps.
However, this requires multiple calls to the model, which may be expensive in terms of cost and time.
Also model may not always identify the correct steps to solve the task, hence is not deterministic.

# Conclusion
Chain of Thought Reasoning is a powerful technique that allows LLMs to solve complex tasks in logical steps. However, it requires multiple calls to the model and may not always identify the correct steps to solve the task. This technique can be useful when the task requires multi-step reasoning or logical thinking.

Care should be taken to ensure that the model is guided correctly and that the output is coherent and accurate.